# Tarea 2: usar SageMaker Feature Store

## Tarea 2.1: opciones de Feature Store

Existen tres formas principales de almacenar funciones en Amazon SageMaker:
1. Usar Amazon SageMaker Feature Store como destino de Amazon SageMaker Data Wrangler después de completar los pasos de preprocesamiento y de agregar las funciones
2. Exportar un cuaderno de SageMaker Data Wrangler que se ejecute a través de la definición de funciones, la creación del grupo de funciones y la ingesta de datos a SageMaker Feature Store
3. Usar el SDK para Python de SageMaker en un cuaderno personalizado que se ejecute a través de la definición de funciones, la creación del grupo de funciones y la ingesta de datos a SageMaker Feature Store

Cada una de estas tres opciones se describe en las siguientes secciones.

### Use SageMaker Feature Store como un destino de SageMaker Data Wrangler

Para agregar SageMaker Feature Store como un destino en Amazon SageMaker Studio, use la opción **Add destination** (Agregar destino). Cuando complete los pasos de preprocesamiento en SageMaker Data Wrangler, puede seleccionar **Add destination** (Agregar destino) en su flujo. SageMaker Studio lo guiará para crear grupos de funciones y le mostrará cómo completar los pasos necesarios para ingerir sus datos preprocesados a SageMaker Feature Store.

Consulte [Cree funciones en Amazon Sagemaker sin código y almacénelas con facilidad](https://aws.amazon.com/blogs/machine-learning/easily-create-and-store-features-in-amazon-sagemaker-without-code/#save_features_to_feature_store) para obtener más información sobre cómo agregar SageMaker Feature Store como un destino.

Consulte [Use Amazon SageMaker Feature Store con Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-use-with-studio.html) para obtener más información sobre cómo crear un grupo de funciones en RSageMaker Studio.

### Exporte un cuaderno desde SageMaker Data Wrangler

Para crear grupos de funciones, use la opción **Export to** (Exportar a) en SageMaker Studio. Cuando usa esta opción, puede crear un cuaderno que incluya todos los comandos necesarios para crear un grupo de funciones.

Con algunas personalizaciones, puede ejecutar el cuaderno para hacer lo siguiente:
- Crear una definición de función según su conjunto de datos
- Crear un grupo de funciones usando la definición de función
- Almacenar el grupo de funciones en SageMaker Feature Store
- Configurar las entradas y salidas de un trabajo de procesamiento

Este laboratorio es similar al cuaderno exportado. Se centra en la primera parte del cuaderno exportado. Muestra cómo ingerir datos al grupo de funciones y extraer registros de un almacén en línea y un almacén sin conexión.

### Use el SDK para Python de SageMaker en un cuaderno personalizado

En este laboratorio, cree un grupo de funciones y extraiga registros de un almacén en línea y un almacén sin conexión. Aprenda cómo funciona SageMaker Feature Store con un cuaderno personalizado. Configure el entorno en la Tarea 2. Luego, complete las siguientes tareas:

Tarea 2.3: configurar SageMaker Feature Store
- Cree funciones en un archivo de cuaderno.
- Cree un grupo de funciones en SageMaker Feature Store.
- Confirme que el grupo de funciones se haya creado.
- Visualice el grupo de funciones en SageMaker Studio.

Tarea 2.4: consultar los almacenes en línea y sin conexión
- Ingiera datos al grupo de funciones.
- Extraiga registros de un almacén en línea.
- Extraiga registros de un almacén sin conexión con Amazon Athena.

## Tarea 2.2: configuración del entorno

Instale los paquetes y las dependencias.

In [ ]:
#install-dependencies

import boto3
import json
import pandas as pd
import sagemaker
import sagemaker_datawrangler
import time
import uuid
import random
from sagemaker.session import Session
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup


region = boto3.Session().region_name
sess = boto3.Session(region_name=region)
bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()

Importe el conjunto de datos procesados del cliente.

In [ ]:
#explore-dataset

column_list = ['income','age','education','education_num','capital_gain','capital_loss','hours_per_week','sex','workclass','marital_status','occupation','relationship','race']
lab_test_data = pd.read_csv('adult_data_processed.csv', names=(column_list), header=1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)

Luego, revise una muestra del conjunto de datos en una tabla.

In [ ]:
#view-dataset

lab_test_data.dtypes
lab_test_data.head()

## Tarea 2.3: configurar SageMaker Feature Store

Cree funciones para entrenar su modelo. Extraiga los datos procesados de su laboratorio de Amazon SageMaker Data Wrangler para crear un grupo de funciones con SageMaker Feature Store.

- Cree funciones en un archivo de cuaderno.
- Cree un grupo de funciones en SageMaker Feature Store.
- Confirme que el grupo de funciones se haya creado.
- Visualice el grupo de funciones en SageMaker Studio.

## Tarea 2.3.1: crear funciones en un archivo de cuaderno

Para crear un grupo de funciones, necesita columnas para asignar las funciones **record_identifier_name** e **event_time_feature_name**. Para cumplir con este requisito, agregue las columnas **record** e **event_time** al conjunto de datos.
- **record_identifier_name** hace referencia a uno de los nombres de una función definida en las definiciones de funciones del grupo. En este laboratorio, cree una columna de ID únicos llamada **record**.
- **event_time_feature_name** es un momento determinado en el que ocurre un nuevo evento que corresponde a la creación o actualización de un registro en un grupo de funciones. Todos los registros del grupo de funciones deben tener un horario de evento correspondiente. Se puede usar para hacer un seguimiento de los cambios en un registro conforme avanza el tiempo. En este laboratorio, cree una columna llamada **event_time**.

Consulte [Introducción a Amazon SageMaker Feature Store](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-concepts.html) para obtener más información sobre el nombre del identificador de registros o el horario del evento.

In [ ]:
#add-required-columns

# Add record and event_time columns
current_time_sec = int(round(time.time()))
lab_test_data.insert(0, 'record', range(0, 0 + len(lab_test_data)))
lab_test_data.insert(1, 'event_time', [current_time_sec]*len(lab_test_data))
lab_test_data['record'] = lab_test_data['record'].astype('string')
lab_test_data['event_time'] = lab_test_data['event_time'].astype('float64')

# Set the record-and-event_time-feature-names
record_identifier_feature_name = 'record'
event_time_feature_name = 'event_time'

# View the dataset
print(lab_test_data.dtypes)
lab_test_data.head()

Puede agregar funciones nuevas en cualquier momento durante el procesamiento de datos según sus necesidades, ya sea con el SDK para Python de SageMaker o con SageMaker Studio.

En este laboratorio, como parte de los pasos del preprocesamiento de datos, use el SDK para Python de SageMaker para agregar una función. Esta función es una combinación ponderada de dos columnas en el conjunto de datos y permite que el modelo se entrene de forma más eficiente.

Cree una función de viabilidad combinando las columnas age y hours_per_week para identificar a los clientes que están más adelantados en sus carreras.

In [ ]:
#add-feature

lab_test_data = lab_test_data.assign(
    workability = 0.5*lab_test_data.age + 0.5*lab_test_data.hours_per_week)

lab_test_data.head()

### Tarea 2.2: crear un grupo de funciones en SageMaker Feature Store

Para ingerir funciones a SageMaker Feature Store, primero establezca las definiciones (nombre de la función y tipo de datos) de todas las funciones que pertenecen al grupo.

Una sola función corresponde a una columna de su conjunto de datos. Un grupo de funciones es un esquema predefinido para una recopilación de funciones. Cada función del grupo tiene un tipo de datos y un nombre especificados. Un solo registro de un grupo de funciones corresponde a una fila de su DataFrame. Un almacén de funciones es una recopilación de grupos de funciones. 

Consulte [Cree, almacene y comparta funciones con Amazon SageMaker Feature Store](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html) para obtener más información sobre SageMaker Feature Store.

Para comenzar el proceso de crear una definición de función, mencione el esquema para cada función.

In [ ]:
#list-column-schemas

column_schemas = [
    {
        "name": "record",
        "type": "string"
    },
    {
        "name": "event_time",
        "type": "float"
    },
    {
        "name": "income",
        "type": "string"
    },
    {
        "name": "age",
        "type": "long"
    },
    {
        "name": "education",
        "type": "float"
    },
    {
        "name": "education_num",
        "type": "float"
    },
    {
        "name": "capital_gain",
        "type": "long"
    },
    {
        "name": "capital_loss",
        "type": "long"
    },
    {
        "name": "hours_per_week",
        "type": "long"
    },
    {
        "name": "sex",
        "type": "float"
    },
    {
        "name": "workclass",
        "type": "array"
    },
    {
        "name": "marital_status",
        "type": "array"
    },
    {
        "name": "occupation",
        "type": "array"
    },
    {
        "name": "relationship",
        "type": "array"
    },
    {
        "name": "race",
        "type": "array"
    },
    {
        "name": "workability",
        "type": "float"
    }
]


Ahora que ya definió el esquema, cree la entrada para las definiciones de las funciones. Configure un mapeo de tipos para los valores long y float del conjunto de datos de FRACTIONAL e INTEGRAL.

Luego, cree la definición de su función estableciendo los valores para **feature_name** y **feature_type** para todas las columnas del esquema que definió.

In [ ]:
#create-feature-definitions

default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

feature_definitions = [
    FeatureDefinition(
        feature_name=column_schema['name'], 
        feature_type=column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in column_schemas
]

En SageMaker Feature Store, los grupos de funciones pueden ser del tipo “solo en línea”, “solo sin conexión” o ambos. En este laboratorio, usa almacenes en línea y sin conexión, de modo que **enable_online_store** debe estar configurado en **True**.
- El almacén en línea está diseñado principalmente para brindar compatibilidad con predicciones en tiempo real que necesitan lecturas con baja latencia de milisegundos y escrituras de alto rendimiento.
- El almacén sin conexión está diseñado principalmente para las predicciones de lotes y el entrenamiento de modelos. Además, es un almacén solo para anexar y se puede usar para almacenar datos históricos de funciones y acceder a ellos.

Cuando su almacén en línea tiene ambas configuraciones, “sin conexión” y “en línea”, cualquier función que ingiera al almacén en línea se replicará en el almacén sin conexión. 

Para configurar el grupo de funciones, especifique las opciones para los siguientes parámetros de configuración:
- **feature_group_name**: es el nombre del grupo de funciones.
- **feature_store_offline_s3_uri**: es la ubicación del bucket de Amazon Simple Storage Service (Amazon S3) en el que SageMaker Feature Store escribe datos en el almacén sin conexión de un grupo de funciones.
- **enable_online_store**: controla si hay o no un almacén en línea habilitado.

In [ ]:
#configure-feature-store

# flow name and a unique ID for this export (used later as the processing job name for the export)
flow_name = "DataWranglerLab"
flow_export_id = f"{time.strftime('%d-%H-%M-%S', time.gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"
feature_group_name = f"FG-{flow_name}-{str(uuid.uuid4())[:8]}"
print(f"Feature Group Name: {feature_group_name}")

feature_store_offline_s3_uri = 's3://' + bucket
enable_online_store = True

Ahora que ya configuró su almacén de funciones, configure una región de AWS e inicie una sesión. Luego, configure un cliente de SageMaker y un tiempo de ejecución para el almacén de funciones. Por último, configure la sesión del almacén de funciones.

Al configurar una sesión del almacén de funciones, se definen los valores **boto_session**, **sagemaker_client** y **sagemaker_featurestore_runtime_client**.

In [ ]:
#set-up-sagemaker-feature-store-session

sagemaker_client = sess.client(service_name='sagemaker', region_name=region)
featurestore_runtime = sess.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=sess,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

Inicialice el grupo de funciones usando los parámetros que configuró antes e invoque la API de Feature Store para crear el grupo de funciones.

In [ ]:
#initialize-feature-group

feature_group = FeatureGroup(
    name=feature_group_name, sagemaker_session=feature_store_session, feature_definitions=feature_definitions)

feature_group.create(
    s3_uri=feature_store_offline_s3_uri,
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=enable_online_store
)

### Tarea 2.3.3: confirmar que el grupo de funciones se haya creado

Su grupo de funciones ya debería estar listo. Confirme que el grupo de funciones se haya creado correctamente.

Espere hasta que el grupo esté listo. Para ello, use la API Describe. Esta función revisa la respuesta que devolvió la API Describe y espera a que el estado sea **Created** (Creado).

In [ ]:
#wait-for-describe

def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"FeatureGroup {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group=feature_group)

Mencione los grupos de funciones usando la API ListFeatureGroups.

In [ ]:
#list-feature-groups

response = sagemaker_client.list_feature_groups()
print(json.dumps(response, indent=4, sort_keys=True, default=str))

### Tarea 2.3.4: visualizar el grupo de funciones en SageMaker Studio

Ya creó un grupo de funciones de SageMaker usando el SDK para Python de SageMaker. Ahora, revise el grupo de funciones en SageMaker Studio para descubrir detalles adicionales.

1. En la pestaña de navegación que se encuentra a la izquierda de su cuaderno, seleccione el ícono **SageMaker Home** (Inicio de SageMaker).

El próximo paso abre una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra Para crear una vista de pantalla dividida de la ventana principal de SageMaker Studio, arrastre la pestaña **lab_5.ipynb** hacia el costado o seleccione (con el botón derecho del mouse) la pestaña **lab_5.ipynb** y elija la opción **New View for Notebook** (Nueva vista para el cuaderno). Ahora puede tener las instrucciones a la vista mientras explora el grupo de funciones.
- **Opción 2**: alternar entre las pestañas de SageMaker Studio para seguir estas instrucciones. Cuando haya terminado de explorar el grupo de funciones, vuelva al cuaderno. Para ello, seleccione la pestaña **lab_5.ipynb**.

2. Expanda la sección **Data** (Datos) y seleccione **Feature Store** (Almacén de funciones).
3. El grupo de funciones que acaba de crear se muestra en la pestaña Feature Store. Puede revisar información detallada sobre el grupo de funciones. Para buscar más detalles, seleccione el grupo de funciones que comienza con **FG-DataWranglerLab-**. Examine los siguientes detalles mientras explora SageMaker Feature Store en SageMaker Studio.
    - **Features** (Funciones): describe todas las funciones en su grupo de funciones, incluido el **tipo** y el momento en que se creó la función según la columna **event_time**.
    **Details** (Detalles): describe los metadatos para el grupo de funciones, incluido el **estado del grupo de funciones**, el **identificador de registros** que configuró antes en el cuaderno, el **tipo de almacén** que estableció en En línea/Sin conexión y el **nombre de la tabla** que puede usar para consultar datos desde el almacén de funciones sin conexión con Athena.
    **Sample Query** (Consulta de muestra): proporciona varias consultas de muestra que puede usar para consultar datos desde el almacén de funciones sin conexión.

## Tarea 2.4: consultar los almacenes en línea y sin conexión

Ya creó un grupo de funciones. Ahora, ingiera datos al grupo de funciones y extraiga registros de un almacén en línea y de un almacén sin conexión con Athena.

- Ingiera datos al grupo de funciones.
- Extraiga registros de un almacén en línea.
- Extraiga registros de un almacén sin conexión con Athena.

### Tarea 2.4.1: ingerir datos al grupo de funciones

Después de crear el grupo de funciones, coloque datos en él. Use **ingest()** para utilizar la llamada a la API PutRecord con el SDK para Python de SageMaker. Cada vez que cree un grupo de funciones por primera vez o que quiera agregar más registros, ingiera los registros al grupo de funciones. 

Para este conjunto de datos, la ingesta demora entre 3 y 5 minutos. Cuando la celda está completa, se muestra una salida como la siguiente:

**IngestionManagerPandas(feature_group_name='FG-DataWranglerLab-13ee4f26', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fdb7fccee60>, sagemaker_session=<sagemaker.session.Session object at 0x7fdb82a900d0>, max_workers=1, max_processes=1, profile_name=None, _async_result=None, _processing_pool=None, _failed_indices=[])**

In [ ]:
#ingest-records

feature_group.ingest(data_frame=lab_test_data, wait=True)

### Tarea 2.4.2: extraer registros de un almacén en línea

El almacén en línea es especialmente útil para las tareas de inferencia porque le permite mostrar un subconjunto de funciones rápidamente.

Ahora que ya se ingirieron sus datos, use **get_record** para extraer un registro de un almacén en línea.

In [ ]:
#get-record

record = random.randint(0, len(lab_test_data.index)-1)
sample_record = featurestore_runtime.get_record(FeatureGroupName=feature_group_name, RecordIdentifierValueAsString=str(record))

print(json.dumps(sample_record, indent=4, sort_keys=True, default=str))

Ahora, use **batch_get_record** para obtener varios registros del grupo de funciones. Ya se seleccionaron varios registros para usted, pero puede cambiar los registros que figuran en **records_list**.

In [ ]:
#batch-get-record

records_list = ["7789", "5646", "309", "24528"]

batch_records = featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": feature_group_name,
            "RecordIdentifiersValueAsString": records_list,
        }
    ]
)

print(json.dumps(batch_records, indent=4, sort_keys=True, default=str))

### Tarea 2.4.3 extraer registros de un almacén sin conexión con Athena

Ahora que ya extrajo registros de un almacén en línea, use Athena para extraer registros de un almacén sin conexión. 

Puede consultar el conjunto de datos completo cuando entrene y ajuste su modelo, o bien consultar un subconjunto de registros para la inferencia. Como SageMaker Feature Store retiene el horario de un evento para cada registro, puede entrenar modelos con exactamente el mismo conjunto de funciones de un horario específico en el pasado sin el riesgo de incluir datos que no correspondan a ese horario. 

¿Cómo adaptaría la consulta para cambiar el subconjunto de datos que obtuvo del almacén sin conexión?

Primero, seleccione la configuración de la consulta. Puede personalizar su consulta para ver todos los subconjuntos de sus datos almacenados en el grupo de funciones. La siguiente consulta es una definición de consulta SELECT (SELECCIONAR) básica.

In [ ]:
#query-settings
# Confirm the Athena settings are configured
try:
    boto3.client('athena').update_work_group(
        WorkGroup='primary',
        ConfigurationUpdates={
            'EnforceWorkGroupConfiguration':False
        }
    )
except Exception:
    pass

#Create the query
query = feature_group.athena_query()
table = query.table_name
query_string = f'SELECT * FROM "{table}" '
output_location = f's3://{bucket}/query_results/'

print(f'Athena query output location: \n{output_location}')

Cuando termine de configurar las opciones, ejecute la consulta y muestre los resultados en forma de tabla.

In [ ]:
#run-athena-query

query.run(query_string=query_string, output_location=output_location)
query.wait()
df = query.as_dataframe()
df.head()

### Conclusión

¡Felicitaciones! Usó SageMaker Feature Store para crear definiciones de funciones para un grupo de funciones en SageMaker Studio y con el SDK para Python de SageMaker. Con su grupo de funciones recién creado, está listo para entrenar y adaptar su modelo con SageMaker Experiments en el siguiente laboratorio. Más adelante en el curso, SageMaker Feature Store le servirá para complementar datos para solicitudes de inferencias debido a la funcionalidad GetRecord de baja latencia que usa el almacén en línea. Seguirá trabajando con este conjunto de datos de ingresos de clientes en el próximo laboratorio.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Conclusión**.